In [1]:
from collections import deque, defaultdict

## Part 1

In [2]:
test = [
"#.#####################",
"#.......#########...###",
"#######.#########.#.###",
"###.....#.>.>.###.#.###",
"###v#####.#v#.###.#.###",
"###.>...#.#.#.....#...#",
"###v###.#.#.#########.#",
"###...#.#.#.......#...#",
"#####.#.#.#######.#.###",
"#.....#.#.#.......#...#",
"#.#####.#.#.#########v#",
"#.#...#...#...###...>.#",
"#.#.#v#######v###.###v#",
"#...#.>.#...>.>.#.###.#",
"#####v#.#.###v#.#.###.#",
"#.....#...#...#.#.#...#",
"#.#########.###.#.#.###",
"#...###...#...#...#.###",
"###.###.#.###v#####v###",
"#...#...#.#.>.>.#.>.###",
"#.###.###.#.###.#.#v###",
"#.....###...###...#...#",
"#####################.#",
]

In [3]:
def create_paths(grid):
    
    paths = defaultdict(set)

    n_row, n_col = len(grid), len(grid[0])
    
    for r, row in enumerate(grid):
        for c, v in enumerate(row):
            if v == ".":
                for d_row, d_col in [(1,0),(0,1),(-1,0),(0,-1)]:
                    next_row, next_col = r + d_row, c + d_col
                    
                    if (min(next_row, next_col) < 0
                       or next_row >= n_row
                       or next_col >= n_col):
                        continue

                    if grid[next_row][next_col] == ".":
                        paths[(r, c)].add((next_row, next_col))
                        paths[(next_row, next_col)].add((r, c))
                        
            if v == ">":
                paths[(r, c)].add((r, c + 1))
                paths[(r, c - 1)].add((r, c))
            
            if v == "v":
                paths[(r, c)].add((r + 1, c))
                paths[(r - 1, c)].add((r, c))

    return paths

In [4]:
def get_path_lengths(grid, paths, start, end):

    n, m = len(grid), len(grid[0])
    steps = 0
    
    q = [(start, steps)]
    visited = set()
    results = []
    
    while q:
        (row, col), steps = q.pop()
        
        if steps == -1:
            visited.remove((row, col))
            continue
            
        if (row, col) == end:
            results.append(steps)
            continue
            
        if (row, col) in visited:
            continue
            
        visited.add((row, col))
        q.append(((row, col), -1))
        
        for next_row, next_col in paths[(row, col)]:
            q.append(((next_row, next_col), steps + 1))
    
    return results

In [5]:
start = (0,1)
end = (len(test)-1,len(test[0])-2)

paths = create_paths(test)
results = get_path_lengths(test, paths, start, end)
assert max(results) == 94

In [6]:
text = open("../advent_of_code_input/2023/day_23.txt", "r").readlines()
text = [i.split("\n")[0] for i in text]

In [7]:
start = (0,1)
end = (len(text)-1,len(text[0])-2)

paths = create_paths(text)
results = get_path_lengths(text, paths, start, end)
max(results)

2190

## Part 2

Following this [approach](https://www.reddit.com/r/adventofcode/comments/18oy4pc/comment/kekhj7f/?utm_source=share&utm_medium=web2x&context=3)

In [8]:
def create_paths(grid):
    
    paths = defaultdict(set)

    n_row, n_col = len(grid), len(grid[0])
    steps = 1

    for r, row in enumerate(grid):
        for c, v in enumerate(row):
            if v in ".>v":
                for d_row, d_col in [(1,0),(0,1),(-1,0),(0,-1)]:
                    next_row, next_col = r + d_row, c + d_col
                    
                    if (min(next_row, next_col) < 0
                       or next_row >= n_row
                       or next_col >= n_col):
                        continue

                    if grid[next_row][next_col] in ".>v":
                        paths[(r, c)].add((next_row, next_col,1))
                        paths[(next_row, next_col)].add((r, c,1))
                        
    return paths

In [9]:
def contract_paths(paths):

    while True:

        for node, edge in paths.items():

            if len(edge) == 2:
                edge_1, edge_2 = edge

                paths[edge_1[:2]].remove(node + (edge_1[2],))
                paths[edge_2[:2]].remove(node + (edge_2[2],))

                paths[edge_1[:2]].add((edge_2[0],edge_2[1],edge_1[2]+edge_2[2]))
                paths[edge_2[:2]].add((edge_1[0],edge_1[1],edge_1[2]+edge_2[2]))
                del paths[node]
                break

        else:
            break
    
    return paths

In [10]:
def get_path_lengths(grid, paths, start, end):

    n, m = len(grid), len(grid[0])
    steps = 0
    
    q = [(start, steps)]
    visited = set()
    results = []
    
    while q:
        (row, col), steps = q.pop()
        
        if steps == -1:
            visited.remove((row, col))
            continue
            
        if (row, col) == end:
            results.append(steps)
            continue
            
        if (row, col) in visited:
            continue
            
        visited.add((row, col))
        q.append(((row, col), -1))
        
        for next_row, next_col,n_steps in paths[(row, col)]:
            q.append(((next_row, next_col), steps + n_steps))
    
    return results

In [11]:
start = (0,1)
end = (len(test)-1,len(test[0])-2)

paths = create_paths(test)
paths = contract_paths(paths)
results = get_path_lengths(test, paths, start, end)

assert max(results) == 154

In [12]:
start = (0,1)
end = (len(text)-1,len(text[0])-2)

paths = create_paths(text)
paths = contract_paths(paths)
results = get_path_lengths(text, paths, start, end)
max(results)

6258